<a href="https://colab.research.google.com/github/geniusrusho/Creating-the-Koch-curve---fractral-geometry/blob/main/Welcome_to_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
Paragraph = "This is NLP session"

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [31]:
sentences =nltk.sent_tokenize(Paragraph)

In [7]:
nltk.word_tokenize(Paragraph)

['This', 'is', 'NLP', 'session']


# ***Implementation OF stemming ***



In [10]:
from nltk.stem import PorterStemmer

In [11]:
stemmer=PorterStemmer()

In [14]:
stemmer.stem('history')

'histori'

## **SENTIMENT ANALYSIS**

In [16]:
from nltk.stem import WordNetLemmatizer

In [17]:
lemmatizer=WordNetLemmatizer()

In [22]:
import nltk 
nltk.download('omw-1.4')
lemmatizer.lemmatize("history")

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


'history'

In [24]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [30]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True